# Curves
Curves are abundant in nueroscience. In particular curves you will typically see are exponential decay, sigmoid, logarithmic and linear. Exponential decays occur in PSC/PSPs (both rise and decay), membrane taus (current clamp), and membrane currents (Ih-currents). Sigmoid currents occur in dose-response curves (FI curves and Ih currents). Logarithmic curves occur in membrane vorage changes in spiking activity in cells. Linear curves are seen in IV curves. In neuroscince many of these curves are measuring some sort of dose-response or response-time relationship.

You have seen these curves in action in several chapters such as the [Current Clamp](current_clamp) and [m/sEPSC](mininature_psc). In this chapter we will delve into the specific parameters of the curves and how the curve_fit function in Scipy works.

Some quick basics that all curves typically have. Many curves go from 0 to 1 or 1 to 0. However, most data we collect is not scaled like this. Curves typically have some scaling factor which means you divide or multiple the equation by a number to change the range of possible the equation can output. Curves also have shifting factors which are added or subtracted to the equation and move the equation up or down or to the left or  right. This is really a core concept that I did not pick up early on but wish I had.

In [3]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import Checkbox, ColumnDataSource, CustomJS, Select, Slider, Spinner
from bokeh.plotting import figure
from scipy import  optimize

# output_notebook()

## Exponential decay
Exponential decay occurs in processes where a value decreases proportionally to its current value. Exponential decay is primarily parameterized by the decay constant, $\lambda$. $\lambda$ can be describe as a time constant, $\tau$ where $\tau = \frac{1}{\lambda}$. $\tau$, the time constant or the time if takes for the process to reach a value of ~1/3 or $\frac{1}{e}$ of the original value. The basic exponential equation is: $N_{0}e^{-\frac{t}{\tau}}$. $N_{0}$ is the original value, and in electrophysiological terms is the amplitude of a PSC/PSP. The equation expects some things. One is that you value decay towards zero. The second is that typically $N_{0}$ is at time point zero. When we curve fit the mEPSCs in the [m/sEPSC](mininature_psc) chapter we created a new time array that started at zero.

The decay equation can be scaled and shifted in several ways. $N_{0}$ can be positive or negative. $N_{0}$ is multiplicative scaling factor, since you leave it out of the equation and you will just get 1 for time value zero since $e^0 = 1$. $\tau$ is a also just a multiplicative scaling factor but for time. You can shift the whole equation up or down by adding a constant if your decay does not converge towards zero but some other constant value. There is no way to just shift the equation along in time. That is because you are assumming $N_{0}$ is $time = 0$ and if you shift $t$ in the equations you change where you are measuring the decay from. To shift the equation in time you just add a constant to your x values after putting them through the equation. You can also add decay equations together to get double, triple or even more decays. However, adding more than 2 or 3 decays together can lead to over fitting and decreased interpretability.

### Single exponential decay

In [ ]:
def exp_decay(x, amplitude=1, tau=1, yshift=0):
    y = yshift+(amplitude * np.exp(-x / tau))
    return y


x = np.arange(300)/10
source = ColumnDataSource({"x": x, "y": exp_decay(x)})

plot = figure(width=400, height=400)

plot.line(x, exp_decay(x), color="black")
plot.line("x", "y", source=source, line_width=3, line_alpha=0.6, line_color="magenta")

yshift = Slider(start=-10, end=10, value=0, step=0.25, title="Y shift")
decay_tau = Slider(start=0.75, end=50, value=1, step=0.25, title="Decay tau")
amplitude = Slider(start=-30, end=30, value=1, step=0.5, title="Amplitude")
length = Slider(start=10, end=70, value=30, step=0.25, title="Length")

callback = CustomJS(
    args=dict(
        source=source,
        decay_tau=decay_tau,
        amplitude=amplitude,
        length=length,
        yshift=yshift,
    ),
    code="""
    const len = Math.round(length.value * 10)
    const t_length = Array.from({ length: len }, (_, i) => (0 + i)/10)
    const y = t_length.map(x => {
        return (amplitude.value * Math.exp((-x) / decay_tau.value))+yshift.value
    })
    source.data.x = t_length;
    source.data.y = y;
    source.change.emit();
""",
)

yshift.js_on_change("value", callback)
decay_tau.js_on_change("value", callback)
amplitude.js_on_change("value", callback)
length.js_on_change("value", callback)

show(row(plot, column(decay_tau, amplitude, length, yshift)))

### Double exponential decay
You will see that you can get much more complicated shapes with a double exponential decay. Some shapes don't even look like a regular decay. There are a couple ways you can parameterize the double exponential decay. You could have a y_shift for each decay or just one for both. Usually when we think a curve is a double exponential decay we are assuming that the amplitude of both decays has the same sign. Additionally, you could make the amplitude the same for each decay or allow them to be different. It starts to get very complicated. You can have any combination of factors you choose. For fitting PSC/PSP decays I usually include an amplitude and tau parameter for each decay but ensure that the amplitudes both have the same sign. In the example below I do not ensure the amplitudes are the same sign. I also encourage you to reparameterize the equation if you feel confident enough.

In [26]:
def db_decay(x, amplitude_fast=1, tau_fast=1, amplitude_slow=0, tau_slow=1):
    y = (amplitude_slow * np.exp(-x / tau_slow)) + (
        amplitude_fast * np.exp(-x / tau_fast)
    )
    return y


x = np.arange(300) / 10
source_db = ColumnDataSource({"x": x, "y": db_decay(x)})

plot = figure(width=400, height=400)

plot.line(x, db_decay(x), color="black")
plot.line("x", "y", source=source_db, line_width=3, line_alpha=0.6, line_color="magenta")

tau_slow = Slider(start=0.75, end=50, value=1, step=0.25, title="Tau slow")
amplitude_slow = Slider(start=-30, end=30, value=1, step=0.5, title="Amplitude_slow")
tau_fast = Slider(start=0.75, end=50, value=1, step=0.25, title="Tau fast")
amplitude_fast = Slider(start=-30, end=30, value=0, step=0.5, title="Amplitude fast")
length = Slider(start=10, end=70, value=30, step=0.25, title="Length")

callback = CustomJS(
    args=dict(
        source=source_db,
        tau_slow=tau_slow,
        amplitude_slow=amplitude_slow,
        tau_fast=tau_fast,
        amplitude_fast=amplitude_fast,
        length=length,
    ),
    code="""
    const len = Math.round(length.value * 10)
    const t_length = Array.from({ length: len }, (_, i) => (0 + i)/10)
    const y = t_length.map(x => {
        return ((amplitude_slow.value * Math.exp((-x) / tau_slow.value))
        +(amplitude_fast.value * Math.exp((-x) / tau_fast.value)))
    })
    source.data.x = t_length;
    source.data.y = y;
    source.change.emit();
""",
)

tau_slow.js_on_change("value", callback)
amplitude_slow.js_on_change("value", callback)
tau_fast.js_on_change("value", callback)
amplitude_fast.js_on_change("value", callback)
length.js_on_change("value", callback)

show(row(plot, column(tau_slow, amplitude_slow, tau_fast, amplitude_fast, length)))

## Sigmoid curve